# Stats Parallel

Notebook by Cascade Tuholske 2021.02.15 <br>
Trying to speed up 4_Event_Stats.py

In [1]:
#### Dependencies
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
import glob
from statistics import mean
import julian
import time 
import multiprocessing as mp 
from multiprocessing import Pool
import os
import sys

In [2]:
#### Step 1 - Function Loads all Tmax Data as an X-array
def read_data(dir_path, space_dim, time_dim):
    """ Function reads in all Tmax .csv files, joins them by date along the x-axis
    and returns the whole record as a x-array data array
    
    Args:   
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    fn_list = sorted(glob.glob(dir_path+'*.csv'))
    df_out = pd.DataFrame()
    date_list = []

    # Open all Tmax files and concat into a df
    for i, fn in enumerate(fn_list):    
        # Open the CSV
        df = pd.read_csv(fn)

        # Get the city ids 
        if i == 1:
            df_id = df[space_dim]

        # get only the Tmax columns and concate date list 
        df_temp = df.iloc[:,3:] # get only temp columns
        date_list = date_list+list(df_temp.columns)

        # Drop cities w/ no temp record 
        df_temp_drop = df_temp.dropna()

        # Merge
        df_out = pd.concat([df_out, df_temp_drop], axis=1)
        print(df_out.shape)
    
    # make date into an array
    tmax_arr = df_out.to_numpy()

    # Make data into an xr.DataArray
    tmax_xr_da = xr.DataArray(tmax_arr, coords=[df_id, date_list], 
                             dims=[space_dim, time_dim])
    return tmax_xr_da

In [3]:
#### Step 2 Function finds all the Tmax Events and writes it to a dateframe w/ dates for each city
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # #CPT 2020.02.23 
        # dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    # df_out['total_days'] = dayTot_list #CPT 2020.02.23
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out


In [4]:
#### Step 3 Function splits the dataset into Tmax events (continuous days >Tmax) for each city
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

def event_split(dates, ID_HDC_G0, intensity, tmax): #, total_days): #CPT 2020.02.23
    
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city

    """

    # city id
    city_id = ID_HDC_G0
    # tot_days = total_days #CPT 2020.02.23
    
    # lists to fill
    city_id_list = []
    # tot_days_list = [] #CPT 2020.02.23
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    year_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = np.sum(intense) # total intensity during event 
        
        start = counter # reset start to current end (e.g. counter)
        year = event_dates[0].split('.')[0]
        
        # fill lists
        city_id_list.append(city_id)
        year_list.append(year)
        # tot_days_list.append(tot_days) #CPT 2020.02.23
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['year'] = year_list
    # df_out['total_days'] = tot_days_list #CPT 2020.02.23
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

#### Step 4 function feeds output from function 3 into function 4
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats

        NOTE - If you add arguments to event_split to make more states,
        be sure to update this function

        args:
            df: input dataframe
    """
    df_out = pd.DataFrame()

    # NOTE - If you add arguments to event_split to make more stats,
    # be sure to update this function

    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        # total_days = row['total_days'] # get total number of tmax days -- CPT 2020.02.23

        df = event_split(dates, ID_HDC_G0, intensity, tmax)# , total_days) #CPT 2020.02.23

        df_out = df_out.append(df)

    return df_out

#### Step 5 function threads it all together
def run_stats(dir_path, space_dim, time_dim, Tthresh, fn_out):
    
    """ Function ties all the Tmax Stats functions together and writes final stats for each Tmax 
    event to a .csv file. Returns results as a dataframe if needed
    
    Args:
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
        Tthresh = float of temp threshold
        fn_out = file and path to write final csv
        
    """
    
    # read in data
    step1= read_data(dir_path, space_dim = space_dim, time_dim = time_dim)
    #step1_sub = step1[:,:10] # subset data for testing
    print('Stack x-array made')
    
    # Mask data based on Tmax threshold ... we're using 40.6C
    step2 = tmax_days(step1, Tthresh)
    print('Tmax masked')
    
    
    # Calculate stats
    step3 = tmax_stats(step2)
    print('Stats made')

    # Save file out
    step3.to_json(fn_out, orient = 'split')
    
    return step3

    print('done')

In [5]:
## Arges Needed 
DATA_IN = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/HI/' # output from avg temp
DATA_OUT = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/'
dir_path = DATA_IN 
space_dim = 'ID_HDC_G0'
time_dim = 'date'
Tthresh = 40.6
fn_out = DATA_OUT+'STATS_1DAY406.json'

In [6]:
# read in data
step1 = read_data(dir_path, space_dim = space_dim, time_dim = time_dim)

(13067, 365)
(13067, 731)
(13067, 1096)
(13067, 1461)
(13067, 1826)
(13067, 2192)
(13067, 2557)
(13067, 2922)
(13067, 3287)
(13067, 3653)
(13067, 4018)
(13067, 4383)
(13067, 4748)
(13067, 5114)
(13067, 5479)
(13067, 5844)
(13067, 6209)
(13067, 6575)
(13067, 6940)
(13067, 7305)
(13067, 7670)
(13067, 8036)
(13067, 8401)
(13067, 8766)
(13067, 9131)
(13067, 9497)
(13067, 9862)
(13067, 10227)
(13067, 10592)
(13067, 10958)
(13067, 11323)
(13067, 11688)
(13067, 12053)
(13067, 12419)


In [7]:
# Mask data based on Tmax threshold ... we're using 40.6C
step2 = tmax_days(step1, Tthresh)
print('Tmax masked')

Tmax masked


In [ ]:
cpu = 20+2 # - 2
n = int(len(step1)/ cpu)  #chunk row size
list_df = [step2 [i:i+n] for i in range(0,step2.shape[0],n)]
print(len(list_df))

In [ ]:
# write them out
for i, df in enumerate(list_df):
    df.to_json(DATA_OUT+'temp/temp_'+str(i)+'.json', orient = 'split')

In [ ]:
#os.mkdir(DATA_OUT+'temp')

In [ ]:
fns_list = glob.glob(DATA_OUT+'temp/*.json')

In [ ]:
len(fns_list)

In [ ]:
fn = fns_list[0]
df = pd.read_json(fn, orient = 'split')

In [ ]:
step3 = tmax_stats(df)

In [ ]:
def max_stats_run(fn):
    
    """ runs max_stats on a fn (.json) and writes on .json)
    Args:
        fn = file name
    """
    
    # open df
    df = pd.read_json(fn, orient = 'split')
    i = fn.split('temp_')[1].split('.json')[0]
    
    # make small for testing 
    df = df.iloc[0:4,:]
    
    # Calculate stats
    step3 = max_stats(df)

    # write file
    fn_out = DATA_OUT+'temp/'+'STAT_'+str(i)+'.json'
    step3.to_json(fn_out, orient = 'split')
    print('done', i)



In [ ]:
for fn in fns_list:
    tbd(fn)

In [ ]:
test = glob.glob(DATA_OUT+'temp/STAT*')

In [ ]:
test

In [ ]:
import Event_Stats_Funcs

In [8]:
test = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_2.json'

In [15]:
data = test.split('_temp/')[1].split('_')[0]
i = test.split(data+'_temp/')[1].split(data+'_')[1]
DATA_OUT = test.split(data+'_temp')[0]

In [16]:
DATA_OUT

'/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/'

In [19]:
glob.glob('/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/*')

['/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_17.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_13.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_2.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_STAT_6.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_STAT_17.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_STAT_2.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_6.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_STAT_13.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_3.json',
 '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_STAT_7.json',
 '/home/cascade/projects/UrbanHeat/data/i

In [21]:
fn = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_STAT_6.json'
test = pd.read_json(fn, orient = 'split')
test.head()

,ID_HDC_G0,year,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax
0,11672,1983,3,43.392558,2.792558,8.377673,"[1983.05.20, 1983.05.21, 1983.05.22]","[0.26288508780000003, 6.2903672981, 1.8244202846]","[40.8628850878, 46.8903672981, 42.424420284600..."
1,11672,1983,3,44.483477,3.883477,11.650432,"[1983.05.30, 1983.05.31, 1983.06.01]","[0.8841091198000001, 2.5332932401, 8.233029562]","[41.4841091198, 43.1332932401, 48.833029562]"
2,11672,1983,1,41.659813,1.059813,1.059813,[1983.06.11],[1.0598134856],[41.6598134856]
3,11672,1983,1,46.931499,6.331499,6.331499,[1983.06.24],[6.3314989252],[46.9314989252]
4,11672,1983,1,51.300997,10.700997,10.700997,[1983.07.03],[10.7009972746],[51.3009972746]
